In [1]:
from src.datasets.kitti_dataset import KittiDataset

dataset_path = "dataset/"  # kitti
sequence = "00"
image_instances_path = "pipeline/vfm-labelss/sam/00/"  # images processed by the SAM algorithm in npz format
kitti = KittiDataset(dataset_path, sequence, image_instances_path)

In [2]:
# setting parameter values

from src.services.preprocessing.common.config import ConfigDTO

config = ConfigDTO(
    **{
        "dataset": kitti,
        "start_index": 20,
        "end_index": 24,
        "start_image_index_offset": 0,
        "alpha_physical_distance": 5,
        "beta_instance_distance": 5,
        "T_normalized_cut": 0.025,
        "reduce_detail_int_to_union_threshold": 0.5,
        "reduce_detail_int_to_mask_threshold": 0.6,
        "cam_name": "cam2",
        "R": 18,
        "nb_neighbors": 25,
        "std_ratio": 5.0,
        "voxel_size": 0.25,
    }
)

In [3]:
# pcd initialisation and initial segmentation based on images

from src.services.preprocessing.init.map import InitMapProcessor
from src.services.preprocessing.init.instances_matrix import InitInstancesMatrixProcessor

init_pcd = InitMapProcessor().process(config)
points2instances = InitInstancesMatrixProcessor().process(config, init_pcd)

In [4]:
points2instances.shape

(498379, 4)

In [5]:
# visualisation of the initial pcd segmentation masks for a particular image

import copy

from src.utils.pcd_utils import color_pcd_by_labels
from src.utils.pcd_utils import visualize_pcd

colored_pcd = color_pcd_by_labels(copy.deepcopy(init_pcd), points2instances[:, 0])
visualize_pcd(colored_pcd)

In [6]:
# pcd handler initialisation

from src.services.preprocessing.not_zero import SelectionNotZeroProcessor
from src.services.preprocessing.in_cube import SelectionInCubeProcessor
from src.services.preprocessing.statistical_outlier import StatisticalOutlierProcessor

processors = [
    SelectionNotZeroProcessor(),
    SelectionInCubeProcessor(),
    StatisticalOutlierProcessor(),
]

In [7]:
# pcd processing and saving the state

import copy

pcd = copy.deepcopy(init_pcd)
for processor in processors:
    pcd, points2instances, indices = processor.process(config, pcd, points2instances)

pcd_for_clustering = copy.deepcopy(pcd)
points2instances_pcd_for_clustering = copy.deepcopy(points2instances)

In [8]:
# visualisation of the processed pcd before voxelization

import copy

from src.utils.pcd_utils import color_pcd_by_labels
from src.utils.pcd_utils import visualize_pcd

colored_pcd_for_clustering = color_pcd_by_labels(copy.deepcopy(pcd_for_clustering), points2instances_pcd_for_clustering[:, 0])
visualize_pcd(colored_pcd_for_clustering)

In [9]:
# final processing step - voxelisation of the pcd

from src.services.preprocessing.voxel_down import VoxelDownProcessor

pcd, points2instances, trace = VoxelDownProcessor().process(config, pcd, points2instances)

In [10]:
# visualisation of the voxelised pcd

import copy

from src.utils.pcd_utils import color_pcd_by_labels
from src.utils.pcd_utils import visualize_pcd

colored_voxel_pcd_for_clustering = color_pcd_by_labels(copy.deepcopy(pcd), points2instances[:, 0])
visualize_pcd(colored_voxel_pcd_for_clustering)

In [11]:
# calculation of distance matrix for voxelised pcd

import numpy as np

from scipy.spatial.distance import cdist
from src.utils.distances_utils import sam_label_distance

points = np.asarray(pcd.points)
spatial_distance = cdist(points, points)

dist, masks = sam_label_distance(
    points2instances,
    spatial_distance,
    3,
    config.beta_instance_distance,
    config.alpha_physical_distance
)

In [12]:
dist.shape

(3502, 3502)

In [13]:
# distance matrix processing

from src.services.distance.isolated import RemovingIsolatedPointsProcessor
from src.services.distance.connected_component import ExtractionLargestConnectedComponentProcessor

distance_processors = [
    RemovingIsolatedPointsProcessor(),
    ExtractionLargestConnectedComponentProcessor(),
]

for processor in distance_processors:
    dist, points, trace = processor.process(dist, points, trace)

In [14]:
print(dist.shape)
print(len(points))
print(len(trace))

(3502, 3502)
3502
3502


In [15]:
# pcd clustering using GraphCut algorithm

from src.services.normalized_cut_service import normalized_cut

eigenval = 2
clusters = normalized_cut(
    dist,
    np.array([i for i in range(len(points))], dtype=int),
    config.T_normalized_cut,
    eigenval
)

In [16]:
len(clusters)

22

In [17]:
clusters[0]

array([ 206,  207,  456,  457,  496,  498, 1271, 1272, 1275, 1276, 1285,
       1390, 1450, 1451, 1559, 1560, 1596, 1597, 2310, 2311])

In [18]:
# visualisation of segmentation results. masks will be drawn on the processed pcd before voxelisation

from src.utils.pcd_utils import color_pcd_by_clusters_and_voxels
from src.utils.pcd_utils import visualize_pcd

pcd_colored = color_pcd_by_clusters_and_voxels(pcd_for_clustering, trace, clusters)
visualize_pcd(pcd_colored)